In [1]:
#PACKAGES

import pandas as pd
import yfinance as yf
import talib
import numpy as np
import requests
from io import StringIO
from datetime import date, datetime, timedelta
import praw
import emoji
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt

In [2]:
#DATAFRAME

rating = pd.read_csv("../Data/rating.csv")
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0"}
html = requests.get(url, headers=headers).text
sp500 = pd.read_html(StringIO(html))[0]

sp500["Symbol"] = sp500["Symbol"].replace({
    "BF.B": "BF-B",
    "BRK.B": "BRK-B"
})

df = pd.merge(
    rating,
    sp500[["Symbol", "Security", "GICS Sector", "GICS Sub-Industry"]],
    on="Symbol",
    how="left"
)

df = df.rename(columns={
    "Security": "Company",
    "GICS Sector": "Sector",
    "GICS Sub-Industry": "Sub-Sector"
})

sector_type_map = {
    "Information Technology": "Growth",
    "Communication Services": "Growth",
    "Consumer Discretionary": "Growth",

    "Health Care": "Defensive",
    "Consumer Staples": "Defensive",
    "Utilities": "Defensive",

    "Financials": "Cyclical",
    "Industrials": "Cyclical",
    "Materials": "Cyclical",
    "Energy": "Cyclical",

    "Real Estate": "Special"
}

symbols = df["Symbol"].unique().tolist()
data = yf.download(symbols, period="5d")["Close"]
latest_close = data.ffill().iloc[-1]
df["Price"] = df["Symbol"].map(latest_close)
eur_usd = yf.Ticker("EURUSD=X").history(period="1d")["Close"].iloc[-1]
df["Price"] = df["Price"] / eur_usd
df["Price"] = df["Price"].round(0).astype(int)

df["Type"] = df["Sector"].map(sector_type_map)
df = df[["Symbol", "Company", "Type", "Sector", "Sub-Sector", "Price", "Fundamental", "Technical", "Sentimental", "Rating"]]
df = df.sort_values(by="Symbol", ascending=True).reset_index(drop=True)

/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_26823/151786377.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbols, period="5d")["Close"]
[*********************100%***********************]  495 of 495 completed


In [3]:
#PORTFOLIO

budget = 5000

target_allocation = {
    'Growth': 0.50,
    'Defensive': 0.30,
    'Cyclical': 0.20
}

sector_weights = {
    #growth
    'Information Technology': 0.40,
    'Communication Services': 0.40,
    'Consumer Discretionary': 0.20,
    #defensive
    'Health Care': 0.40,
    'Consumer Staples': 0.40,
    'Utilities': 0.20,
    #cyclical
    'Financials': 0.40,
    'Industrials': 0.20,
    'Materials': 0.20,
    'Energy': 0.20
}

sector_budgets = {}

for typ, type_pct in target_allocation.items():
    type_budget = budget * type_pct
    type_sectors = df[df['Type'] == typ]['Sector'].unique()
    type_sector_weights = {s: sector_weights[s] for s in type_sectors if s in sector_weights}
    total_weight = sum(type_sector_weights.values())
    
    for sector, weight in type_sector_weights.items():
        sector_budgets[sector] = type_budget * (weight / total_weight)

portfolio = []

for sector, sector_budget in sector_budgets.items():
    sector_df = df[df['Sector'] == sector].nlargest(20, 'Rating')  
    
    allocated = 0
    for _, stock in sector_df.iterrows():
        if allocated + stock['Price'] <= sector_budget * 1.1: 
            portfolio.append({
                'Symbol': stock['Symbol'],
                'Company': stock['Company'],
                'Type': stock['Type'],
                'Sector': stock['Sector'],
                'Sub-Sector': stock['Sub-Sector'],
                'Rating': stock['Rating'],
                'Price': stock['Price'],
                'Shares': 1
            })
            allocated += stock['Price']
        if allocated >= sector_budget * 0.9:  
            break

portfolio_df = pd.DataFrame(portfolio)
portfolio_df['Investment'] = portfolio_df['Price'] * portfolio_df['Shares']
total_investment = portfolio_df['Investment'].sum()

print(f"Total invested: {total_investment}€")
print(f"Number of stocks: {len(portfolio_df)}")

Total invested: 5080€
Number of stocks: 38


In [5]:
#SAVE 

df.to_csv("../Data/analysis.csv", index=False)
portfolio_df.to_csv("../Data/portfolio.csv", index=False)